In [1]:
import os
import time
import pandas as pd
import OpenDartReader
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import time
import re

In [4]:
# 환경변수 및 DART API 설정
load_dotenv(dotenv_path='.env_dart')
api_key = os.getenv('DART_API_KEY')
dart = OpenDartReader(api_key)

# 종목코드 정제

In [5]:
# 엑셀 파일 로드
excel_file = '경쟁률종목정보_데이터확인_20251223.xlsx'
df_origin = pd.read_excel(excel_file, header=2).copy()
df_origin

FileNotFoundError: [Errno 2] No such file or directory: '경쟁률종목정보_데이터확인_20251223.xlsx'

In [ ]:
# 종목코드 정제
df_origin['종목코드_정제']=df_origin['종목코드'].astype(str).str.strip().str.upper().str.zfill(6)
df_origin['종목코드_정제']

In [ ]:
# 컬럼 제목 행 제거
df_origin = df_origin[df_origin['종목코드_정제']!='STOCK_CD']
df_origin

In [ ]:
# 주간사 정보 정제
df_origin['주간사코드']=df_origin['주간사코드'].astype(str).str.strip()
df_origin['주간사명']=df_origin['주간사명'].astype(str).str.strip()

In [ ]:
# DART에 등록된 모든 회사의 고유번호 리스트 가져오기
corp_codes = dart.corp_codes

# 종목코드로 고유번호 찾기 예시
target_corp = corp_codes[corp_codes['stock_code'] == '005930']
real_corp_code = target_corp.iloc[0]['corp_code'] # 여기서 8자리 번호가 나옵니다.

In [ ]:
# 1. 먼저 DART 시스템의 모든 고유번호 리스트를 가져와서 'corps' 변수에 담습니다.
# (이 줄이 빠지면 NameError가 발생합니다!)
corps = dart.corp_codes 

# 2. 고유번호를 저장할 빈 컬럼을 만듭니다.
df_origin['고유번호'] = ''

# 3. 반복문을 돌며 6자리 종목코드로 8자리 고유번호를 찾습니다.
for idx, row in df_origin.iterrows():
    target_code = row['종목코드_정제']
    
    # 전체 리스트(corps)에서 해당 종목코드와 일치하는 행을 필터링합니다.
    matched = corps[corps['stock_code'] == target_code]
    
    if not matched.empty:
        # 일치하는 게 있다면 8자리 고유번호(corp_code)를 가져옵니다.
        actual_corp_code = matched.iloc[0]['corp_code']
        df_origin.at[idx, '고유번호'] = actual_corp_code
        print(f"매칭 성공: {row['종목명']} -> {actual_corp_code}")
    else:
        # 상장사가 아니거나 코드가 다를 경우
        print(f"매칭 실패: {row['종목명']} ({target_code})")

# 4. 상위 5개 결과 확인
df_origin[['종목명', '종목코드_정제', '고유번호']].head()


In [ ]:
test_corp = df_origin[df_origin['고유번호'] != ''].iloc[0]['고유번호']
print("테스트용 corp_code =", test_corp)

test_df = dart.list(
    test_corp, 
    start='2015-01-01',
    end='2026-12-31'
)

print(type(test_df))
print(test_df.columns)
print(test_df.head())



In [ ]:
results = []

for idx, row in df_origin.iterrows():
    corp_code = row['고유번호']
    stock = row['종목코드_정제']
    name = row['종목명']

    if not corp_code:
        results.append([stock, name, None, None])
        continue

    try:
        # 검색 시작일을 넉넉히 설정
        data = dart.list(corp_code, start='2015-01-01', end='2026-12-31')

        rcept_no = None
        report_nm = None

        if data is not None and not data.empty:
            # 증권발행실적보고서만 필터링
            target = data[data['report_nm'].str.contains("증권발행실적보고서", na=False)]

            if not target.empty:
                # [수정포인트] iloc[0](최신) 대신 iloc[-1](최초) 선택
                first_report = target.iloc[-1] 
                rcept_no = first_report['rcept_no']
                report_nm = first_report['report_nm']
                print(f"✅ {name} (최초공시) → {rcept_no}")
            else:
                print(f"⚠️ {name} : 증권발행실적보고서 없음")
        else:
            print(f"⚠️ {name} : 공시 없음")

        results.append([stock, name, rcept_no, report_nm])

    except Exception as e:
        print(f"🚨 오류 {name}: {e}")
        results.append([stock, name, None, None])

df_rcept = pd.DataFrame(results, columns=["종목코드","종목명","rcept_no","report_nm"])

### -> https://dart.fss.or.kr/dsaf001/main.do?rcpNo= 뒤에다가 방금 구한 종목코드(EX.20251201000400) 집어넣고 웹브라우저 검색 돌리면 바로 그 종목 증권발행실적 보고서 뜸